# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [29]:
#url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
#s = requests.get(url).content
#df=pd.read_csv(io.StringIO(s.decode('UTF8')))
### Если не работает загрузка on-line
df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


https://datatofish.com/if-condition-in-pandas-dataframe/ Материалы для решения

In [30]:
# Парсим столбец Норматив
dt=df['Норматив'].str.split(' ', n = 3, expand = True)
df['Норм1'] = dt[2].str.strip(" ")
dt1=df['Норм1'].str.split('-', n = 3, expand = True)
df['Норм2'] = dt1[0].str.strip(" ")
df['Норм3'] = dt1[1].str.strip(" ")

In [31]:
df

,Показатель,Единица измерений,Результат анализа,Норматив,Норм1,Норм2,Норм3
0,pH,единицы pH,8.4,в пределах 6-9,6-9,6,9
1,Запах,баллы,1,не более 2-3,2-3,2,3
2,Цветность,градусы,б/цвета,не более 30,30,30,None
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,7-10,7,10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5","1,5","1,5",None
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3","3,3","3,3",None
6,Нитраты (по NO3),мг/дм3,24,не более 45,45,45,None
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5","3,5","3,5",None
8,Хлориды (Cl),мг/дм3,200,не более 350,350,350,None
9,Сульфаты (SO4),мг/дм3,189.5,не более 500,500,500,None


In [52]:
#Создаем столбец result с заполненеим его с помощью отбора по трем условиям
df.loc[(df.replace('б/цвета', '0', regex=True).replace('/.', ',', regex=True)['Результат анализа'].astype(float)>df.replace(',', '.', regex=True)['Норм2'].astype(float)), 'result'] = df['Норм2']+'<='+df['Показатель']
df.loc[(df.replace('б/цвета', '0', regex=True).replace('/.', ',', regex=True)['Результат анализа'].astype(float)>df.replace(',', '.', regex=True)['Норм2'].astype(float))&(df.replace('б/цвета', '0', regex=True).replace('/.', ',', regex=True)['Результат анализа'].astype(float)<df.replace(',', '.', regex=True)['Норм3'].astype(float)), 'result'] = df['Норм2']+'<='+df['Показатель']+'<='+df['Норм3']
df.loc[(df.replace('б/цвета', '0', regex=True).replace('/.', ',', regex=True)['Результат анализа'].astype(float)<df.replace(',', '.', regex=True)['Норм2'].astype(float)), 'result'] = df['Показатель']+'<='+df['Норм2']
#Делаем столбец Показатель индексным
df=df[['Показатель','Единица измерений','Результат анализа','Норматив','result']].set_index('Показатель')


In [53]:
df

,Единица измерений,Результат анализа,Норматив,result
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,6<=pH<=9
Запах,баллы,1,не более 2-3,Запах<=2
Цветность,градусы,б/цвета,не более 30,Цветность<=30
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,7<=Жёсткость<=10
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5","Аммиак и аммоний-ион (по азоту)<=1,5"
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3","Нитриты (по NO2)<=3,3"
Нитраты (по NO3),мг/дм3,24,не более 45,Нитраты (по NO3)<=45
Фосфаты (P),мг/дм3,0.36,"не более 3,5","Фосфаты (P)<=3,5"
Хлориды (Cl),мг/дм3,200,не более 350,Хлориды (Cl)<=350


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


https://www.dmitrymakarov.ru/python/random-11/ для изучения

In [60]:
# Ваше решение
box=['A','A','A','A','A','Y','Y','Y','Y']

In [127]:
#выборка без возвращения (с возвращением 0,17):
ishod=[]
for i in range (0,100000):
    ishod.append(list(np.random.choice(box, 3, replace = False)))

In [128]:
P_black = ishod.count (['A', 'A', 'A'])/len(ishod)
P_black

0.11895

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [ ]:
# Ваше решение

In [132]:
det=['S','S','S','S','S','S','S','N','N','N']

In [133]:
ishod2=[]
for i in range (0,100):
    ishod2.append(list(np.random.choice(det, 2, replace = False)))
print(ishod2)

[['S', 'S'], ['N', 'S'], ['S', 'S'], ['S', 'S'], ['S', 'N'], ['S', 'N'], ['N', 'S'], ['N', 'S'], ['N', 'N'], ['S', 'S'], ['S', 'S'], ['N', 'S'], ['N', 'S'], ['S', 'N'], ['S', 'S'], ['S', 'S'], ['N', 'S'], ['N', 'S'], ['N', 'S'], ['S', 'S'], ['S', 'S'], ['S', 'N'], ['S', 'N'], ['S', 'S'], ['S', 'S'], ['S', 'S'], ['S', 'S'], ['S', 'S'], ['S', 'N'], ['N', 'S'], ['S', 'N'], ['N', 'S'], ['N', 'S'], ['N', 'S'], ['S', 'S'], ['S', 'N'], ['N', 'S'], ['S', 'S'], ['S', 'S'], ['N', 'S'], ['N', 'S'], ['N', 'S'], ['S', 'N'], ['S', 'S'], ['N', 'N'], ['S', 'N'], ['S', 'S'], ['S', 'S'], ['S', 'S'], ['S', 'S'], ['S', 'S'], ['N', 'S'], ['N', 'S'], ['S', 'S'], ['N', 'S'], ['S', 'S'], ['S', 'S'], ['S', 'S'], ['S', 'N'], ['S', 'S'], ['S', 'N'], ['S', 'S'], ['N', 'S'], ['S', 'N'], ['S', 'S'], ['N', 'N'], ['S', 'S'], ['S', 'S'], ['S', 'S'], ['S', 'N'], ['S', 'S'], ['S', 'S'], ['S', 'S'], ['S', 'N'], ['N', 'S'], ['N', 'S'], ['S', 'S'], ['S', 'S'], ['S', 'S'], ['S', 'S'], ['S', 'N'], ['N', 'S'], ['S', 'N'], ['N

In [134]:
P = ishod2.count (['S', 'N'])/len(ishod2)
P

0.23